<h1>Eigenvalues in the complex plane</h1>
<p>
    This file provides code to visualise the eigenvalues of the linear Douglas-Rachford method as described in Section 4.1.<br />
    Especially Figure 4.1 may be reproduced using the provided code below.
</p>
<p>
    The preconditioned Douglas-Rachford iteration for $A,B,\Delta\in\mathbb R^{n\times n}$ is given as the fixed point iteration of
</p>
<p>
    $$
        \begin{split}
            H_\Delta &= J_{\Delta A}(\Delta A + J_{\Delta B} (\mathbb I - \Delta A))\\
            &= (\mathbb I + \Delta A + \Delta B + \Delta B \Delta A)^{-1}(\mathbb I + \Delta B \Delta A),
        \end{split}
    $$
</p>
<p>
    where $\mathbb I\in\mathbb R^{n\times n}$ is the identity matrix. Furthermore, assume that all requirements on $A, B$ and $\Delta$ from Lemma 4.1 are fulfilled. 
</p>
<p>
    In the following, code for three possible choices of $\Delta$ may be found:
</p>
<ol>
    <li> $\Delta = t \mathbb I$ for $t >0$.<br /></li>
    <li> $\Delta = \begin{bmatrix}t \mathbb I & 0 \\ 0 & s \mathbb I\end{bmatrix}$ with $t, s >0$.<br /></li>
</ol>
<p>
    &nbsp;
<p>
    We start with general settings and definitions for all three choices:
</p>

In [1]:
# import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, SelectionSlider
from IPython.display import display

In [2]:
# dimension of the matrices A, B and Delta.
# has to be even for 2nd choice of Delta!
n = 50

In [3]:
# in order to recreate the images from Figure 4.1 set the seed to 100
np.random.seed(100)

In [4]:
# create random maximal monotone (i.e. positive semi-definite) matrices A and B

A_build = np.random.uniform(-10.0, 10.0, (n, n))
B_build = np.random.uniform(-10.0, 10.0, (n, n))

A = A_build.T @ A_build
B = B_build.T @ B_build

I = np.identity(n)

In [5]:
# preparations to plot a circle in the complex plane
# (i.e. the circle described in Lemma 4.1 in which all eigenvalues lie)

theta = np.linspace(0, 2*np.pi, 100)

circle_x = 0.5 * np.cos(theta) + 0.5 
circle_y = 0.5 * np.sin(theta)

<hr />
<h3>
    1. $\Delta = t\mathbb I$:
</h3>
<p>
    We start precomputing eigenvalues for different values of $t>0$ first:
</p>

In [6]:
t_range = np.logspace(-4, 1, 100)
eigenvalues_t = []

for t_index, t in enumerate(t_range):
    H_Delta = np.linalg.solve(I + t * A + t * B + t**2 * B @ A, I + t**2 * B @ A)
    
    eigenvalues_t.append(np.linalg.eigvals(H_Delta))

<p>
    Now the eigenvalues can be drawn in the complex plane:
</p>

In [8]:
# create a figure
plt.rcParams['figure.dpi'] = 100
figure, axes = plt.subplots()

# styling figure
plt.xlim([0.0, 1.2])
plt.ylim([-0.55, 0.7])

plt.xticks([1.0])
plt.yticks([-0.5, 0.0, 0.5])

axes.set_aspect(1)
axes.set_xlabel("Re$(\lambda)$", loc='right', labelpad=-5.0)
axes.set_ylabel("Im$(\lambda)$", loc='top', rotation=0, labelpad=-25.0)

ax = plt.gca()
ax.spines['top'].set_color('none')
ax.spines['bottom'].set_position('zero')
ax.spines['left'].set_position('zero')
ax.spines['right'].set_color('none')

ax.plot(1, 0, ">k", transform=ax.get_yaxis_transform(), clip_on=False)
ax.plot(0, 1, "^k", transform=ax.get_xaxis_transform(), clip_on=False)

# plot from Lemma 4.1
axes.plot(circle_x, circle_y, color="black", lw=0.5)

dots = plt.scatter(eigenvalues_t[0].real, eigenvalues_t[0].imag, color='red', s=8, zorder=10)

plt.close()

# update eigenvalues in plot depending on value for t
def plot_eigenvalues(t_index):

    dots.set_offsets([[z.real, z.imag] for z in eigenvalues_t[t_index]])
    display(figure)

slider = SelectionSlider(
    options = [("{:.5f}".format(t), t_index) for t_index, t in enumerate(t_range)],
    description='$t$:'
)

display(interact(plot_eigenvalues, t_index=slider))

interactive(children=(SelectionSlider(description='$t$:', options=(('0.00010', 0), ('0.00011', 1), ('0.00013',…

<function __main__.plot_eigenvalues(t_index)>

<hr />
<h3>
    2. $\Delta = \begin{bmatrix}t \mathbb I & 0 \\ 0 & s \mathbb I\end{bmatrix}$:
</h3>
<p>
    We start precomputing eigenvalues for different values of $t>0$ and $s>0$ first:
</p>

In [9]:
s_range = np.logspace(-4, 1, 100)
eigenvalues_ts = []

for t_index, t in enumerate(t_range):
    eigenvalues_ts.append([])
    for s_index, s in enumerate(s_range):
        
        Delta = np.block([[t * np.identity(int(n/2)), np.zeros((int(n/2), int(n/2)))], [np.zeros((int(n/2), int(n/2))), s * np.identity(int(n/2))]])
        
        H_Delta = np.linalg.solve(I + Delta @ A + Delta @ B + Delta @ B @ Delta @ A, I + Delta @ B @ Delta @ A)
    
        eigenvalues_ts[t_index].append(np.linalg.eigvals(H_Delta))

<p>
    Now the eigenvalues can be drawn in the complex plane:
</p>

In [11]:
# create a figure
plt.rcParams['figure.dpi'] = 100
figure, axes = plt.subplots()

# styling figure
plt.xlim([0.0, 1.2])
plt.ylim([-0.55, 0.7])

plt.xticks([1.0])
plt.yticks([-0.5, 0.0, 0.5])

axes.set_aspect(1)
axes.set_xlabel("Re$(\lambda)$", loc='right', labelpad=-5.0)
axes.set_ylabel("Im$(\lambda)$", loc='top', rotation=0, labelpad=-25.0)

ax = plt.gca()
ax.spines['top'].set_color('none')
ax.spines['bottom'].set_position('zero')
ax.spines['left'].set_position('zero')
ax.spines['right'].set_color('none')

ax.plot(1, 0, ">k", transform=ax.get_yaxis_transform(), clip_on=False)
ax.plot(0, 1, "^k", transform=ax.get_xaxis_transform(), clip_on=False)

# plot from Lemma 4.1
axes.plot(circle_x, circle_y, color="black", lw=0.5)

dots = plt.scatter(eigenvalues_ts[0][0].real, eigenvalues_ts[0][0].imag, color='red', s=8, zorder=10)

plt.close()

# update eigenvalues in plot depending on value for t
def plot_eigenvalues(t_index, s_index):

    dots.set_offsets([[z.real, z.imag] for z in eigenvalues_ts[t_index][s_index]])
    display(figure)

t_slider = SelectionSlider(
    options = [("{:.5f}".format(t), t_index) for t_index, t in enumerate(t_range)],
    description='$t$:'
)

s_slider = SelectionSlider(
    options = [("{:.5f}".format(s), s_index) for s_index, s in enumerate(s_range)],
    description ='$s$:'
)

display(interact(plot_eigenvalues, t_index = t_slider, s_index = s_slider))

interactive(children=(SelectionSlider(description='$t$:', options=(('0.00010', 0), ('0.00011', 1), ('0.00013',…

<function __main__.plot_eigenvalues(t_index, s_index)>